In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [4]:
raw_data = pd.read_csv("../../data_clean.csv")
data = raw_data[raw_data['birth date'] < 2020].copy()

data['age'] = data['parole board interview date'] - data['birth date']
data['jail duration'] = data['parole board interview date'] - data['year of entry']

others_parole_type =  ['PIE', 'SP CONSDR', 'ECPDO', 'MEDICAL','RESCISSION', 'DEPORT']
data['parole board interview type'] = data['parole board interview type'].replace(others_parole_type, 'OTHERS').replace('SUPP MERIT', 'MERIT TIME').replace('PV REAPP', 'REAPPEAR')

data = data.dropna(axis=0, subset=['crime 1 - class', 'parole eligibility date'])
data = data.drop(columns={"sex", "race / ethnicity"})

df_one_hot = pd.get_dummies(data, columns=[
    "crime 1 - class", "crime 2 - class",
    "crime 3 - class", "crime 4 - class",
    "parole board interview type"])

df_one_hot.drop(columns=['release date','birth date', 'year of entry'],inplace=True)

X_train, X_test, y_train, y_test = train_test_split(df_one_hot.drop('y',axis=1), df_one_hot['y'], 
                                                    stratify=df_one_hot['y'], test_size=0.3, random_state=42)

In [5]:
# Create and configure the XGBoost classifier with balanced class weights
param_grid = {
    'n_estimators': [200, 500, 1000],
    'max_depth': [3, 5, 10],
    'learning_rate': [1e-2, 0.1]
}

class_weights = len(y_train) / (2 * np.bincount(y_train))

# Define base estimator (XGB Classifier)
xgb = xgb.XGBClassifier(
    objective='binary:logistic',
    scale_pos_weight=class_weights[1] / class_weights[0],
    random_state=42
)

# Define GridSearchCV
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='f1')

# Fit the GridSearch on the training data and select the best estimator
grid_search.fit(X_train, y_train)
xgb_classifier = grid_search.best_estimator_

# Make predictions on both the training and test datasets
train_predictions = xgb_classifier.predict(X_train)
test_predictions = xgb_classifier.predict(X_test)

# Calculate evaluation metrics for the training dataset
train_accuracy = accuracy_score(y_train, train_predictions)
train_balanced_accuracy = balanced_accuracy_score(y_train, train_predictions)
train_roc_auc = roc_auc_score(y_train, train_predictions)
train_f1 = f1_score(y_train, train_predictions)

# Calculate evaluation metrics for the test dataset
test_accuracy = accuracy_score(y_test, test_predictions)
test_balanced_accuracy = balanced_accuracy_score(y_test, test_predictions)
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_predictions)

# Print the evaluation metrics
print("Training Metrics:")
print(f"Accuracy: {train_accuracy:.2f}")
print(f"Balanced Accuracy: {train_balanced_accuracy:.2f}")
print(f"ROC AUC: {train_roc_auc:.2f}")
print(f"F1 Score: {train_f1:.2f}")
print("\nTest Metrics:")
print(f"Accuracy: {test_accuracy:.2f}")
print(f"Balanced Accuracy: {test_balanced_accuracy:.2f}")
print(f"ROC AUC: {test_roc_auc:.2f}")
print(f"F1 Score: {test_f1:.2f}")

Training Metrics:
Accuracy: 0.69
Balanced Accuracy: 0.72
ROC AUC: 0.72
F1 Score: 0.50

Test Metrics:
Accuracy: 0.66
Balanced Accuracy: 0.68
ROC AUC: 0.68
F1 Score: 0.46
